In [1]:
import pandas as pd

# Especificamos de que tipo son los datos
dtypes = {
  "Date": str,
  "Time": str, 
  "Global_active_power": float,
  "Global_reactive_power": float,
  "Voltage": float,
  "Global_intensity": float,
  "Sub_metering_1": float,
  "Sub_metering_2": float,
  "Sub_metering_3": float
}

Energy = pd.read_csv('household_power_consumption.txt',delimiter=";",dtype=dtypes,na_values="?")

In [2]:
Energy.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [3]:
Energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Date                   object 
 1   Time                   object 
 2   Global_active_power    float64
 3   Global_reactive_power  float64
 4   Voltage                float64
 5   Global_intensity       float64
 6   Sub_metering_1         float64
 7   Sub_metering_2         float64
 8   Sub_metering_3         float64
dtypes: float64(7), object(2)
memory usage: 142.5+ MB


In [4]:
Energy.duplicated().sum()

0

In [6]:
# Tratamos los valores nulos con la mediana de cada fila
for i in Energy.columns[2:]:
    median = Energy[i].median()
    Energy[i].fillna(median, inplace=True)

In [11]:
Energy.isna().sum()

Date                     0
Time                     0
Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

In [9]:
# Convierte la columna 'Date' al formato datetime
Energy['Date'] = pd.to_datetime(Energy['Date'], format='%d/%m/%Y')

# Formatea la columna 'Date' al nuevo formato 'aaaa-mm-dd'
Energy['Date'] = Energy['Date'].dt.strftime('%Y-%m-%d')

In [10]:
Energy.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,2006-12-16,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,2006-12-16,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,2006-12-16,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,2006-12-16,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [ ]:
Energy.to_csv("EnergyConsumption.csv", index=False)

Maejamos los mensajes que le enviamos a la base de datos con el siguiente codigo

In [11]:
import paho.mqtt.client as mqtt
import json
import time

# Conectar con el broker MQTT
client = mqtt.Client()
client.connect("localhost", 1883)

for index, fila in Energy.iloc[243168:].iterrows():
    # Crear un diccionario con los valores de la fila
    mensaje = {
        "Date": fila["Date"],
        "Time": fila["Time"],
        "Gap": fila["Global_active_power"],  # Mapear el nombre de la columna
        "Grp": fila["Global_reactive_power"],  # Mapear el nombre de la columna
        "Voltage": fila["Voltage"],
        "Gi": fila["Global_intensity"],  # Mapear el nombre de la columna
        "Sm1": fila["Sub_metering_1"],  # Mapear el nombre de la columna
        "Sm2": fila["Sub_metering_2"],  # Mapear el nombre de la columna
        "Sm3": fila["Sub_metering_3"]  # Mapear el nombre de la columna
    }
    
    # Convertir el diccionario a formato JSON
    mensaje_json = json.dumps(mensaje)
    
    # Publicar el mensaje en el tema "val1"
    client.publish("val1", mensaje_json)
    time.sleep(0.05)

# Desconectar del broker MQTT
client.disconnect()